In [1]:
#Library
library(shiny)
library(shinythemes)
library(dplyr)
library(shinydashboard)
library(lubridate)
library(cowplot)
library(gridExtra)
library(reshape2)
library(ggplot2)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'shinydashboard'

The following object is masked from 'package:graphics':

    box


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Warning message:
"package 'cowplot' was built under R version 3.6.3"Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'cowplot'

The following object is masked from 'package:lubridate':

    stamp

Warning message:
"package 'gridExtra' was built under R version 3.6.3"
Attaching package: 'gridExtra'

The following object is masked from 'package:dplyr':

    combine



In [2]:
#load data
prod_ranking_model <- read.csv("Product-Rankings.csv", check.names = FALSE)
cust_prod_ranking_model <- read.csv("Customer-Product-Rankings.csv", check.names = FALSE)
cust_correlation_model <- read.csv("Customer-Customer-Correlation-Matrix.csv", row.names = "Party", check.names = FALSE)
prod_correlation_model <- read.csv("Product-Product-Correlation-Matrix.csv", row.names = "Product", check.names = FALSE)

In [3]:
#Load data visualize 
#(1)
data1 <- read.csv("Sales-Transactions-Edited.csv")
data1$Date <- as.Date(data1$Date)
data1$Voucher <- as.numeric(data1$Voucher)
data1$Qty <- as.numeric(data1$Qty)


In [4]:
#1
data1$Month <- format(as.Date(data1$Date), "%m")
monthly_vouchers <- aggregate(Voucher ~ Month, data = data1, sum)
head(monthly_vouchers )

Month,Voucher
01,43044896
02,44709620
03,63599764
04,4302207
05,14382595
06,20952749


In [5]:
#2
data1$Year <- format(as.Date(data1$Date), "%Y")
yearly_vouchers <- aggregate(Voucher ~ Year, data = data1, sum)
head(yearly_vouchers)

Year,Voucher
2017,128356143
2018,183296965
2019,115423409


In [6]:
data1$Month <- as.numeric(data1$Month)
data1$Year <- as.numeric(data1$Year)

In [7]:
#3
year_2017 <- subset(data1, Year == "2017")
monthly_vouchers_2017 <- aggregate(Voucher ~ Month, data = year_2017, sum) %>%
                         mutate(Year = "2017")

year_2018 <- subset(data1, Year == "2018")
monthly_vouchers_2018 <- aggregate(Voucher ~ Month, data = year_2018, sum) %>%
                         mutate(Year = "2018")

year_2019 <- subset(data1, Year == "2019")
monthly_vouchers_2019 <- aggregate(Voucher ~ Month, data = year_2019, sum) %>%
                         mutate(Year = "2019")

monthly_vouchers <- bind_rows(monthly_vouchers_2017, monthly_vouchers_2018, monthly_vouchers_2019)
head(monthly_vouchers)

Month,Voucher,Year
4,1696897,2017
5,6193116,2017
6,9112124,2017
7,10351453,2017
8,16419181,2017
9,17923006,2017


In [8]:
#4 với từng tháng trong năm 2017, 2018, 2019 lấy ra các nhà cung cấp có số lượng sản phẩm nhiều nhất
df_2017 <- data1[data1$Year == 2017,]
max_Party_2017 <- df_2017 %>%
  group_by(Month, Party) %>%
  summarize(total_qty = sum(Qty)) %>%
  group_by(Month) %>%
  filter(total_qty == max(total_qty)) %>%
  mutate(Year = "2017")

df_2018 <- data1[data1$Year == 2018,]
max_Party_2018 <- df_2018 %>%
  group_by(Month, Party) %>%
  summarize(total_qty = sum(Qty)) %>%
  group_by(Month) %>%
  filter(total_qty == max(total_qty))%>%
  mutate(Year = "2018")

df_2019 <- data1[data1$Year == 2019,]
max_Party_2019 <- df_2019 %>%
  group_by(Month, Party) %>%
  summarize(total_qty = sum(Qty)) %>%
  group_by(Month) %>%
  filter(total_qty == max(total_qty))%>%
  mutate(Year = "2019")

max_Party <- bind_rows(max_Party_2017, max_Party_2018, max_Party_2019)
head(max_Party)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Month,Party,total_qty,Year
4,TP13,77109,2017
5,TP13,79595,2017
6,TP13,65179,2017
7,TP13,58336,2017
8,TP13,90284,2017
9,TP13,60551,2017


In [9]:
#5 với từng tháng trong năm 2017, 2018, 2019 lấy ra tên các sản phẩm có số lượng nhiều nhất
df_2017 <- data1[data1$Year == 2017,]

max_products_2017 <- df_2017 %>%
  group_by(Month, Product) %>%
  summarize(total_qty = sum(Qty)) %>%
  group_by(Month) %>%
  filter(total_qty == max(total_qty)) %>%
  mutate(Year = "2017")

df_2018 <- data1[data1$Year == 2018,]

max_products_2018 <- df_2018 %>%
  group_by(Month, Product) %>%
  summarize(total_qty = sum(Qty)) %>%
  group_by(Month) %>%
  filter(total_qty == max(total_qty)) %>%
  mutate(Year = "2018")

df_2019 <- data1[data1$Year == 2019,]

max_products_2019 <- df_2019 %>%
  group_by(Month, Product) %>%
  summarize(total_qty = sum(Qty)) %>%
  group_by(Month) %>%
  filter(total_qty == max(total_qty)) %>%
  mutate(Year = "2019")

max_products <- bind_rows(max_products_2017, max_products_2018, max_products_2019)
head(max_products)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Month,Product,total_qty,Year
4,14 GREEN,195900,2017
5,14 GREEN,299720,2017
6,14 GREEN,229000,2017
7,14 GREEN,125410,2017
8,14 GREEN,271428,2017
9,14 GREEN,143705,2017


In [10]:
#6 Trong năm 2017, 2018, 2019 lấy ra tên 10 sản phẩm có số lượng nhiều nhất
df_2017 <- data1[data1$Year == 2017,]

product_qty_2017 <- df_2017 %>%
  group_by(Product) %>%
  summarize(total_qty = sum(Qty)) %>%
  mutate(Year = "2017")
  top_products_2017 <- product_qty_2017 %>%
  arrange(desc(total_qty)) %>%
  head(10)

df_2018 <- data1[data1$Year == 2018,]

product_qty_2018 <- df_2018 %>%
  group_by(Product) %>%
  summarize(total_qty = sum(Qty)) %>%
  mutate(Year = "2018")
  top_products_2018 <- product_qty_2018 %>%
  arrange(desc(total_qty)) %>%
  head(10)

df_2019 <- data1[data1$Year == 2019,]

product_qty_2019 <- df_2019 %>%
  group_by(Product) %>%
  summarize(total_qty = sum(Qty)) %>%
  mutate(Year = "2019")
  top_products_2019 <- product_qty_2019 %>%
  arrange(desc(total_qty)) %>%
  head(10)

top_products <- bind_rows(top_products_2017, top_products_2018, top_products_2019)
head(top_products)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Product,total_qty,Year
14 GREEN,1615541,2017
CYCLE-BLU-10*12,280880,2017
500ML PLASTIC BOULS,201850,2017
DURGA 10*12 BLUE,185230,2017
10*12 SARAS-NAT,172160,2017
VISHNU 250ML,150645,2017


In [11]:
#7 phần trăm số lượng sản phầm nhiều nhất của 10 nhà cung cấp
Party_qty <- data1 %>%
  group_by(Party) %>%
  summarize(total_qty = sum(Qty)) %>%
  ungroup()

top_party <- Party_qty %>%
  arrange(desc(total_qty)) %>%
  head(n = 10)

party_percent <- data1 %>%
  filter(Party %in% top_party$Party) %>%
  group_by(Party) %>%
  summarize(percent_qty = sum(Qty)/sum(top_party$total_qty)*100)
head(party_percent)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Party,percent_qty
DSP-SEKHAR,4.320300
GANESHMESS,4.470765
HAMPI FOODS,5.822907
K.SRIHARI,12.824069
KPR,9.738151
SEKHAR MARKET,16.544150


In [12]:
#8
top_products <- data1 %>%
group_by(Year, Product) %>%
summarize(total_Rate = sum(Rate)) %>%
arrange(Year, desc(total_Rate)) %>%
group_by(Year) %>%
slice(1:10)
head(top_products)

Year,Product,total_Rate
2017,LITE FOAM(1200),867660.0
2017,LOOSE ITEMS,674479.6
2017,DONA-VAI-9100,666740.0
2017,GST AMOUNT,453943.9
2017,SILV-CONT-750ML(100,388885.0
2017,CLASSIC ENJOY(750),257750.0


In [13]:
#9
df_2017 <- data1[data1$Year == 2017,]
product_rate <- df_2017 %>%
  group_by(Product) %>%
  summarize(total_rate = sum(Rate))
top_products_1 <- product_rate %>%
  arrange(desc(total_rate)) %>%
  head(10)

df_2018 <- data1[data1$Year == 2018,]
product_rate <- df_2018 %>%
  group_by(Product) %>%
  summarize(total_rate = sum(Rate))
top_products_2 <- product_rate %>%
  arrange(desc(total_rate)) %>%
  head(10)

df_2019 <- data1[data1$Year == 2019,]
product_rate <- df_2019 %>%
  group_by(Product) %>%
  summarize(total_rate = sum(Rate))
top_products_3 <- product_rate %>%
  arrange(desc(total_rate)) %>%
  head(10)

top_products_1
top_products_2
top_products_3

Product,total_rate
LITE FOAM(1200),867660.0
LOOSE ITEMS,674479.6
DONA-VAI-9100,666740.0
GST AMOUNT,453943.9
SILV-CONT-750ML(100,388885.0
CLASSIC ENJOY(750),257750.0
SILV-CONT-450ML(150,212150.0
DONA-1100,163455.0
SMARTBOUL GLA(4000),161560.0
GOLD LEAF SITTING,159305.0


Product,total_rate
LOOSE ITEMS,1631853
GST AMOUNT,1057511
DONA-VAI-9100,908080
LITE FOAM(1200),691070
SILV-CONT-750ML(100,547595
SILV-CONT-450ML(150,281235
SMART BOUL(48),221221
GOLD LEAF SITTING,181250
100LEAF -SP,147865
SMARTBOUL-CLAS(40),141200


Product,total_rate
LOOSE ITEMS,2954220.0
DONA-VAI-9100,856927.0
SILV-CONT-750ML(100,562695.0
GST AMOUNT,502425.2
SMART BOUL(48),376874.0
LITE FOAM(1200),370200.0
SILV-CONT-450ML(150,204360.0
GOLD LEAF SITTING,161170.0
SILV-CONT-250ML(200,119680.0
DONA-1100,86200.0


In [14]:
install.packages("lubridate")
library(lubridate)

also installing the dependency 'timechange'




  There are binary versions available but the source versions are later:
           binary source needs_compilation
timechange  0.0.2  0.2.0              TRUE
lubridate  1.7.10  1.9.2              TRUE

  Binaries will be installed


Warning message:
"package 'lubridate' is in use and will not be installed"

package 'timechange' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\yongs\AppData\Local\Temp\RtmpAVsu9p\downloaded_packages


In [15]:
#10
year_2017 <- subset(data1, Year == "2017")
# Chuyển đổi cột Month sang định dạng ngày/tháng/năm
year_2017$Month <- ymd(paste0(year_2017$Year, "-", year_2017$Month, "-01"))

product_summary_1 <- aggregate(Rate ~ Month, data = year_2017, sum)


year_2018 <- subset(data1, Year == "2018")
year_2018$Month <- ymd(paste0(year_2018$Year, "-", year_2018$Month, "-01"))

product_summary_2 <- aggregate(Rate ~ Month, data = year_2018, sum)


year_2019 <- subset(data1, Year == "2019")
year_2019$Month <- ymd(paste0(year_2019$Year, "-", year_2019$Month, "-01"))

product_summary_3 <- aggregate(Rate ~ Month, data = year_2019, sum)

product_summary_1
product_summary_2
product_summary_3

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Month,Rate
2017-04-01,892995.9
2017-05-01,842990.3
2017-06-01,639799.6
2017-07-01,684525.2
2017-08-01,916384.0
2017-09-01,654104.1
2017-10-01,621104.4
2017-11-01,643446.6
2017-12-01,728887.4


Month,Rate
2018-01-01,554312.0
2018-02-01,461266.0
2018-03-01,669300.5
2018-04-01,634541.0
2018-05-01,663427.4
2018-06-01,841297.1
2018-07-01,774831.8
2018-08-01,865094.1
2018-09-01,887152.2
2018-10-01,820764.4


Month,Rate
2019-01-01,807867.3
2019-02-01,873956.3
2019-03-01,1067808.8
2019-04-01,1014670.2
2019-05-01,1076380.1
2019-06-01,897260.8
2019-07-01,907205.3
2019-08-01,774210.2
2019-09-01,674780.2
2019-10-01,175381.6


In [16]:
#11
data_pro_rank_sorted <- arrange(prod_ranking_model, desc(Rate))
top_product1 <- head(data_pro_rank_sorted, 10)
top_product1

data_pro_rank_sorted <- arrange(prod_ranking_model, Rate)
top_product2 <- head(data_pro_rank_sorted, 10)
top_product2

Product,Rate,Top_Sell_Rank,Popularity_Rank
LOOSE ITEMS,200000.0,142,82
CASH,40000.0,795,524
GST AMOUNT,39478.5,499,228
VAT AMOUNT,23096.0,666,365
FOM-PRINT(50*120),6000.0,850,732
STAND BOUL(1200),4700.0,834,732
SILV-CONT-750ML(100,3600.0,127,44
5*5 DONA(4500),3200.0,655,408
"12"" SPARKLE(1050)",3150.0,792,655
"7""CLASSIC(4800)",3120.0,812,704


Product,Rate,Top_Sell_Rank,Popularity_Rank
LIDS-450MW,0.33,135,732
100ML HIPS,1.00,445,732
LIDS-450ML,1.00,26,418
LOCK COVER,1.00,850,732
OWN ITEM,1.00,850,732
CLOTH SEAL COVER,1.20,198,732
ARKA SNACKS PLATES,1.30,278,732
PET WATER GLASS,1.50,249,732
"ARCA 6""",2.00,305,732
ORACLE 300ML FANCY,2.00,445,704


In [17]:
#12
# 10 sản phẩm có độ phổ biến cao nhất lấy ra phần trăm giá của chúng
Product_Popularity_Rank <- arrange(prod_ranking_model, Popularity_Rank)
top_product_Popularity_Rank <- head(Product_Popularity_Rank, 5)

total_rate <- sum(top_product_Popularity_Rank$Rate)
top_rate_pro_popu_Rank_percent_1 <- top_product_Popularity_Rank %>%
  mutate(percent = Rate/total_rate *100)
top_rate_pro_popu_Rank_percent_1

# 10 sản phẩm có độ phổ biến thấp nhất lấy ra phần trăm giá của chúng
Product_Popularity_Rank <- arrange(prod_ranking_model, desc(Popularity_Rank))
top_product_Popularity_Rank <- head(Product_Popularity_Rank, 5)

total_rate <- sum(top_product_Popularity_Rank$Rate)
top_rate_pro_popu_Rank_percent_2 <- top_product_Popularity_Rank %>%
  mutate(percent = Rate/total_rate *100)
top_rate_pro_popu_Rank_percent_2

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Product,Rate,Top_Sell_Rank,Popularity_Rank,percent
14 GREEN,400,1,1,73.800738
TIRUMALA-50(16*20),30,19,2,5.535055
BLACK DOG-350ML,33,15,3,6.088561
APPLE WATER,29,11,4,5.350554
VISHNU 300ML,50,16,5,9.225092


Product,Rate,Top_Sell_Rank,Popularity_Rank,percent
1.25 COOLDRINKS,670,834,732,26.377953
"10"" GREEN",1610,794,732,63.385827
"10"" SILVER HEAVY",25,774,732,0.984252
10*10 JANATHA,135,727,732,5.314961
10*10 PRIYA 2ND,100,727,732,3.937008


In [18]:
#13
data_top_sell_sorted <- arrange(prod_ranking_model, Top_Sell_Rank)
top_sell_rank_1 <- head(data_top_sell_sorted, 10)
top_sell_rank_1

data_top_sell_sorted <- arrange(prod_ranking_model, Top_Sell_Rank)
top_sell_rank_2 <- head(data_top_sell_sorted, 10)
top_sell_rank_2

Product,Rate,Top_Sell_Rank,Popularity_Rank
14 GREEN,400.0,1,1
CYCLE-BLU-10*12,7.1,2,13
500ML PLASTIC BOULS,3.8,3,48
DURGA 10*12 BLUE,110.0,4,50
10*12 SARAS-NAT,81.0,5,38
CYCLE-BK-10*12,13.0,6,34
VISHNU 250ML,35.0,7,6
ROBO 10*12,5.5,8,53
10*12 KRISHNA-BK(10,15.0,9,24
TIRUMALA-50(13*16),28.0,10,9


Product,Rate,Top_Sell_Rank,Popularity_Rank
14 GREEN,400.0,1,1
CYCLE-BLU-10*12,7.1,2,13
500ML PLASTIC BOULS,3.8,3,48
DURGA 10*12 BLUE,110.0,4,50
10*12 SARAS-NAT,81.0,5,38
CYCLE-BK-10*12,13.0,6,34
VISHNU 250ML,35.0,7,6
ROBO 10*12,5.5,8,53
10*12 KRISHNA-BK(10,15.0,9,24
TIRUMALA-50(13*16),28.0,10,9


In [19]:
#14
data_top_sell_sorted <- arrange(prod_ranking_model, desc(Top_Sell_Rank))
top_sell_rank_1 <- head(data_top_sell_sorted, 10)
top_sell_rank_1

data_top_sell_sorted <- arrange(prod_ranking_model, desc(Top_Sell_Rank))
top_sell_rank_2 <- head(data_top_sell_sorted, 10)
top_sell_rank_2


Product,Rate,Top_Sell_Rank,Popularity_Rank
SUPER CLASSIC,120,866,732
10*15 SPECIAL D-CUT,140,850,732
"6"" THERM(5600)",2240,850,732
"7""ESSFOAM(2250)",1575,850,732
CLING FILM 600MTS,500,850,732
FOM-PRINT(50*120),6000,850,732
LOCK COVER,1,850,732
MAAZA TETRA,330,850,732
NAPKEENS PRINT 33*3,50,850,732
NON WOVEN 16*20 -CL,140,850,732


Product,Rate,Top_Sell_Rank,Popularity_Rank
SUPER CLASSIC,120,866,732
10*15 SPECIAL D-CUT,140,850,732
"6"" THERM(5600)",2240,850,732
"7""ESSFOAM(2250)",1575,850,732
CLING FILM 600MTS,500,850,732
FOM-PRINT(50*120),6000,850,732
LOCK COVER,1,850,732
MAAZA TETRA,330,850,732
NAPKEENS PRINT 33*3,50,850,732
NON WOVEN 16*20 -CL,140,850,732


In [20]:
#15
data_pro_rank_sorted <- arrange(cust_prod_ranking_model, desc(Rate))
top_product1 <- head(data_pro_rank_sorted, 10)
top_product1

data_pro_rank_sorted <- arrange(cust_prod_ranking_model, Rate)
top_product2 <- head(data_pro_rank_sorted, 10)
top_product2



Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank
A1 PLASTICS-SPT,LOOSE ITEMS,2e+05,1,19541,1,19516
ACK,LOOSE ITEMS,2e+05,1,19427,1,19403
AJANTHA HOTEL,LOOSE ITEMS,2e+05,1,19364,1,19346
AMMA AGENCIES,LOOSE ITEMS,2e+05,720,19276,2,19272
ANILFANCY,LOOSE ITEMS,2e+05,1,19271,1,19240
ANR-VGIRI,LOOSE ITEMS,2e+05,1,19187,1,19166
BCM,LOOSE ITEMS,2e+05,2,18410,1,18409
BHAGWAN CATERING-NE,LOOSE ITEMS,2e+05,5,18324,1,18329
BRHAMADEVI SUDHEER,LOOSE ITEMS,2e+05,1,18242,1,18185
BSCPL INFRASTRUCTUR,LOOSE ITEMS,2e+05,2,18004,2,18000


Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank
SUNDHA PLASTICS,LIDS-450MW,0.33,10000,3614,1,3604
BALAJI PLASTICS,LIDS-450ML,1.00,15000,18523,1,18517
CUSTOMER,LOCK COVER,1.00,1,16283,1,16194
HAMPI FOODS,LIDS-450ML,1.00,4000,14903,3,14882
K.KONDAYYA-GDR,OWN ITEM,1.00,1,13808,1,13801
K.SRIHARI,LIDS-450ML,1.00,3000,13593,1,13646
KK TRADERS,LIDS-450ML,1.00,500,12721,1,12735
KNC,LIDS-450ML,1.00,6000,12590,1,12629
KVKC,100ML HIPS,1.00,500,11812,1,11921
P.SRINIVASULU-RAILW,LIDS-450ML,1.00,3500,8815,2,8816


In [21]:
#16
data_top_sellcupro_sorted <- arrange(cust_prod_ranking_model, desc(Top_Sell_Rank))
top_sellcuspro_rank_1 <- head(data_top_sellcupro_sorted, 10)
top_sellcuspro_rank_1

data_top_sellcupro_sorted <- arrange(cust_prod_ranking_model, Top_Sell_Rank)
top_sellcuspro_rank_2 <- head(data_top_sellcupro_sorted, 10)
top_sellcuspro_rank_2

Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank
A.BHASKER-VGIRI,LITE FOAM(1200),1750.0,1,19727,1,19715
A.BHASKER-VGIRI,GARUDA 10*12 BLUE,13.5,100,19724,1,19715
A.BHASKER-VGIRI,GARUDA 13*16 BLUE,15.0,100,19724,1,19715
A.BHASKER-VGIRI,PAPER TEA 80ML LONG,36.0,100,19724,1,19715
A.BHASKER-VGIRI,N0-5,160.0,3,19720,1,19715
A.BHASKER-VGIRI,NO-0,1540.0,3,19720,1,19715
A.BHASKER-VGIRI,NO-1,160.0,3,19720,1,19715
A.BHASKER-VGIRI,NO-2,160.0,3,19720,1,19715
A.BHASKER-VGIRI,VISHNU CHOTA WINE,45.0,30,19719,1,19715
A.BHASKER-VGIRI,TIRML-BL-KG-17*23,130.0,4,19718,1,19715


Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank
YVT-PMR,VISHNU 250ML,35,600,1,4,1
YVT-PMR,NO-00,160,5,2,1,2
YVT-PMR,TIRUMALA KIRANA,160,43,3,1,2
YVT-PMR,N0-5,160,10,4,1,2
YVT-PMR,NO-0,1540,10,4,1,2
YVT-PMR,NO-1,160,10,4,1,2
YVT-PMR,NO-2,160,10,4,1,2
YVSR-VGIRI,NO-2,160,9,8,2,8
YVSR-VGIRI,NO-0,1540,8,9,2,8
YVSR-VGIRI,N0-5,160,5,10,1,13


In [22]:
#17
# 10 sản phẩm có độ phổ biến cao nhất lấy ra phần trăm giá của chúng
Product_Popularity_Rank <- arrange(cust_prod_ranking_model, desc(Popularity_Rank))
top_product_Popularity_Rank <- head(Product_Popularity_Rank, 5)

total_rate <- sum(top_product_Popularity_Rank$Rate)
top_rate_pro_popu_Rank_percent_1 <- top_product_Popularity_Rank %>%
  mutate(percent = Rate/total_rate *100)
top_rate_pro_popu_Rank_percent_1


Product_Popularity_Rank <- arrange(cust_prod_ranking_model, Popularity_Rank)
top_product_Popularity_Rank <- head(Product_Popularity_Rank, 5)

total_rate <- sum(top_product_Popularity_Rank$Rate)
top_rate_pro_popu_Rank_percent_2 <- top_product_Popularity_Rank %>%
  mutate(percent = Rate/total_rate *100)
top_rate_pro_popu_Rank_percent_2

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank,percent
A.BHASKER-VGIRI,14 GREEN,400.0,40,19717,1,19715,17.1049818
A.BHASKER-VGIRI,GARUDA 10*12 BLUE,13.5,100,19724,1,19715,0.5772931
A.BHASKER-VGIRI,GARUDA 13*16 BLUE,15.0,100,19724,1,19715,0.6414368
A.BHASKER-VGIRI,LITE FOAM(1200),1750.0,1,19727,1,19715,74.8342955
A.BHASKER-VGIRI,N0-5,160.0,3,19720,1,19715,6.8419927


Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank,percent
YVT-PMR,VISHNU 250ML,35,600,1,4,1,1.703163
YVT-PMR,N0-5,160,10,4,1,2,7.785888
YVT-PMR,NO-0,1540,10,4,1,2,74.939173
YVT-PMR,NO-00,160,5,2,1,2,7.785888
YVT-PMR,NO-1,160,10,4,1,2,7.785888


In [23]:
#18
# 5 sản phẩm có số lượng đơn đặt hàng ở cấp độ khách hàng nhiều nhất
data_cuspro_rank_sorted <- arrange(cust_prod_ranking_model, desc(No_of_Orders))
top_cusproduct <- head(data_cuspro_rank_sorted, 10)
top_cusproduct


Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank
TP13,PP,165,6464,1830,338,1917
TP13,DURGA 10*12 BLUE,110,303350,1966,274,1979
TP13,VISHNU 300ML,50,36330,1938,273,1980
TP13,10*10 TEJA,142,7223,1798,246,1989
TP13,APPLE WATER,29,37788,1933,240,1990
TP13,BLACK DOG-350ML,33,26276,2001,224,1996
TP13,10*12 SARAS-NAT,81,144640,2142,222,1997
TP13,VISHNU 250ML,35,26750,1999,213,2000
TP13,APPLE WINE,30,22035,2044,203,2007
TP13,BAHUBALI WINE,31,24463,2022,194,2057


In [24]:
ui <- shinyUI(dashboardPage(
  dashboardHeader(title = "Product Recommendation", titleWidth = 300),
  dashboardSidebar(
    sidebarMenu(
      menuItem("Home", tabName = "home", icon = icon("home")),
      menuItem("Visualize", tabName = "visualize", icon = icon("chart-bar")),
      menuItem("Recommendation System", tabName = "recommend", icon = icon("thumbs-up"))
    )
  ),
  dashboardBody(
    tabItems(
      tabItem(tabName = "home",
              fluidRow(
                column(width = 6,
                       h3("Information Team:"),
                       textOutput("person1"),
                       textOutput("person2"),
                       textOutput("person3"),
                       textOutput("person4")
                ),
                column(width = 6,
                       h3("Review project:"),
                       HTML("a voluminous roger wool how dearly discourage a tidy band
              \nmover supposing continually command a easy-going 
              \ntool reduction even knottily stake a traumatic 
              \nharass insolence yet optimistically permeate 
              \na wooden realign seed till well coordinate 
              \na foolish castrate wallaby while judgementally 
              \nblemish a adventurous stir jaguar before 
              \npromptly glass a far-flung desensitise swell 
              \nbefore triumphantly prescind a mammoth hesitate snarl.</p>")
                )
              )
      ),
      tabItem(tabName = "visualize",
              fluidRow(
                column(width = 5,
                       selectInput("PlotType", "Select Plot :",
                                   choices = c(
                                               "For each month (months are combined from 3 years) there will be a total of vouchers (1)",
                                               "For each year, there will be a total of voucher values (2)",
                                               "Total Vouchers by Month and Year(3)",
                                               "Top Party with Qty highest by Month and Year(4)",
                                               "Top Products with Qty highest by Month and Year(5)",
                                               "Top 10 Products with Qty highest by Year(6)",
                                               "Top 10 Products by Year(7)",
                                               "Top 10 highest Rate products in 2018(8)",
                                               "Total Rate by Month(9)",
                                               "Top 10 Products by Rate(10)",
                                               "Rate percentage of the 5 products with Popularity_Rank(11)",
                                               "Rate of the 10 products with top_sell_rank(12)",
                                               "Popularity_Rank of the 10 products with top_sell_rank(13)",
                                               "Top 10 Products by Party by Rate (14)",
                                               "Party of 10 products with top_sell_rank(15)",
                                               "Rate percentage of the 5 products with Popularity_Rank(16)",
                                               "10 Products with the highest No_of_Orders at customer level(17)"
                                              ),
                                   selected = "For each month (months are combined from 3 years) there will be a total of vouchers (1)"
                       )
                ),
                column(width = 8,
                       plotOutput("selectedPlot")
                )
              )
      ),
      tabItem(tabName = "recommend",
              fluidRow(
                column(width = 4,
                       tags$h3("Input:"),
                       textInput("txt1", "Enter Customer Name:", ""),
                       actionButton("loginButton", "Login"),
                       textInput("txt2", "Enter Product Name:", ""),
                       actionButton("viewButton", "View")
                ),
                column(width = 8,
                       uiOutput("loggedInUI"),
                       uiOutput("viewedInUI")  
                )
              )
      )
    )
  )
))


In [ ]:
#Server
server <- function(input, output, session) {
  
  #Output tab Home
  output$person1 <- renderText({
    "Nguyen Ba Hoang Nam - HE161029"
  })
  
  output$person2 <- renderText({
    "   Thai Ba Bao      - HE163102 "
  })
  
  output$person3 <- renderText({
    "   Le Van Huynh     - HE163919"
  })
  
  output$person4 <- renderText({
    "   Do Thanh Dat     - HE161567 "
  })   
  
  #Output tab Visualize  
  #Scatter chart 
generate1 <- function() {
  ggplot(monthly_vouchers, aes(x = Month, y = Voucher)) +
      geom_col(fill = "#1c4f6e")+
    labs(title = "For each month there will be a total of vouchers")
}

generate2 <- function() {
    ggplot(yearly_vouchers, aes(x = Year, y = Voucher)) +
      geom_col(fill = "#88e885")+
    labs(title = "For each year there will be a total of voucher values")
}
    
generate3 <- function() {
ggplot(monthly_vouchers, aes(x = Month, y = Voucher, fill = Year)) +
  geom_col(position = "dodge") +
  scale_fill_manual(values = c("#FF7F0E", "#1F77B4", "#2CA02C")) +
  labs(title = "Total Vouchers by Month and Year",
       x = "Month",
       y = "Total Vouchers") +
  theme_bw(base_size = 14, base_family = "Helvetica") +
  theme(plot.title = element_text(hjust = 0.5),
        axis.text.x = element_text(angle = 90, hjust = 1),
        legend.title = element_blank(),
        legend.position = "bottom",
        panel.grid.major = element_line(color = "#EAEAEA"),
        strip.background = element_rect(fill = "#F5F5F5"),
        strip.text = element_text(size = 12)) +
  facet_wrap( ~ Year, ncol = 3)
}
    
generate4 <- function() {
ggplot(max_Party, aes(x = as.factor(Month), y = total_qty, fill = Party)) +
  geom_col(position = "dodge") +
  labs(title = "Top Party with Qty highest by Month and Year",
       x = "Month",
       y = "Total Quantity") +
  theme_bw(base_size = 14, base_family = "Helvetica") +
  theme(plot.title = element_text(hjust = 0.5),
        axis.text.x = element_text(angle = 90, hjust = 1),
        legend.title = element_blank(),
        legend.position = "bottom",
        panel.grid.major = element_line(color = "#EAEAEA"),
        strip.background = element_rect(fill = "#F5F5F5"),
        strip.text = element_text(size = 12)) +
  facet_wrap( ~ Year, ncol = 3)
}
    
generate5 <- function() {
ggplot(max_products, aes(x = as.factor(Month), y = total_qty, fill = Product)) +
  geom_col(position = "dodge") +
  labs(title = "Top Products with Qty highest by Month and Year",
       x = "Month",
       y = "Total Quantity") +
  theme_bw(base_size = 14, base_family = "Helvetica") +
  theme(plot.title = element_text(hjust = 0.5),
        axis.text.x = element_text(angle = 90, hjust = 1),
        legend.title = element_blank(),
        legend.position = "bottom",
        panel.grid.major = element_line(color = "#EAEAEA"),
        strip.background = element_rect(fill = "#F5F5F5"),
        strip.text = element_text(size = 12)) +
  facet_wrap( ~ Year, ncol = 3)
}
    
generate6 <- function() {
ggplot(party_percent, aes(x = "", y = percent_qty, fill = Party)) +
geom_bar(stat = "identity", width = 1) +
geom_text(aes(label = paste0(round(percent_qty, 1), "%")), position = position_stack(vjust = 0.5)) +
coord_polar(theta = "y") +
ggtitle("Product Percentage of Top 10 Party") +
scale_fill_hue(name = "Party") +
scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
theme(axis.text.y = element_blank(),
axis.ticks.y = element_blank(),
axis.title.y = element_blank())
}
    
generate7 <- function() {
ggplot(top_products, aes(x = as.factor(Year), y = total_Rate, fill = Product)) +
  geom_col(position = "stack") +
  labs(title = "Top 10 Products by Year",
       x = "Year",
       y = "Total Quantity") +
  theme_bw(base_size = 14, base_family = "Helvetica") +
  theme(plot.title = element_text(hjust = 0.5),
        axis.text.x = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_blank(),
        panel.grid.major = element_line(color = "#EAEAEA")) +
  guides(fill = guide_legend(reverse = TRUE))
}
    
generate8 <- function() {
plot1 <- ggplot(top_products_1, aes(x = total_rate, y = Product, fill = Product)) +
  geom_col()+
  ggtitle("Top 10 highest Rate products in 2017")
plot2 <- ggplot(top_products_2, aes(x = total_rate, y = Product, fill = Product)) +
  geom_col()+
  ggtitle("Top 10 highest Rate products in 2018")
plot3 <- ggplot(top_products_3, aes(x = total_rate, y = Product, fill = Product)) +
  geom_col()+
  ggtitle("Top 10 highest Rate products in 2019")

plot_grid(plot1, plot2, plot3, ncol=1)
}
    
generate9 <- function() {
plot1 <- ggplot(product_summary_1, aes(x = Month, y = Rate)) +
  geom_line(color = "orange") +
  labs(x = "Month", y = "Total Rate") +
  ggtitle("Total Rate by Month in 2017") +
  scale_x_date(date_labels = "%B")
plot2 <- ggplot(product_summary_2, aes(x = Month, y = Rate)) +
  geom_line(color = "orange") +
  labs(x = "Month", y = "Total Rate") +
  ggtitle("Total Rate by Month in 2018") +
  scale_x_date(date_labels = "%B")
plot3 <- ggplot(product_summary_3, aes(x = Month, y = Rate)) +
  geom_line(color = "orange") +
  labs(x = "Month", y = "Total Rate") +
  ggtitle("Total Rate by Month in 2019") +
  scale_x_date(date_labels = "%B")

plot_grid(plot1, plot2, plot3, ncol=1)
    
}
generate10 <- function() {
plot1 <- ggplot(top_product1, aes(x = Rate, y = Product, fill = Product)) +
  geom_col() +
  ggtitle("Top 10 Products by Rate (Descending)")
plot2 <- ggplot(top_product2, aes(x = Rate, y = Product, fill = Product)) +
  geom_col() +
  ggtitle("Top 10 Products by Rate (Ascending)")

# Ghép các biểu đồ lại với nhau
plot_grid(plot1, plot2, ncol=1)
    
}
generate11 <- function() {
plot_1 <- ggplot(top_rate_pro_popu_Rank_percent_1, aes(x = "", y = percent, fill = Product)) +
  geom_bar(stat = "identity", width = 1) +
geom_text(aes(label = paste0(round(percent, 1), "%")), position = position_stack(vjust = 0.5)) +
coord_polar(theta = "y") +
ggtitle("Rate percentage of the 5 products with the highest Popularity_Rank") +
scale_fill_hue(name = "Product") +
scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
theme(axis.text.y = element_blank(),
axis.ticks.y = element_blank(),
axis.title.y = element_blank())

plot_2 <- ggplot(top_rate_pro_popu_Rank_percent_2, aes(x = "", y = percent, fill = Product)) +
  geom_bar(stat = "identity", width = 1) +
geom_text(aes(label = paste0(round(percent, 1), "%")), position = position_stack(vjust = 0.5)) +
coord_polar(theta = "y") +
ggtitle("Rate percentage of the 5 products with the lowest Popularity_Rank") +
scale_fill_hue(name = "Product") +
scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
theme(axis.text.y = element_blank(),
axis.ticks.y = element_blank(),
axis.title.y = element_blank())

plot_grid(plot_1, plot_2, ncol=1)
}
    
generate12 <- function() {
plot_1 <- ggplot(top_sell_rank_1, aes(x = Top_Sell_Rank, y = Rate , fill = Product)) +
  geom_col()+
  ggtitle("Rate of the 10 products with the lowest top_sell_rank ")

plot_2 <- ggplot(top_sell_rank_2, aes(x = Top_Sell_Rank, y = Popularity_Rank , fill = Product)) +
  geom_col()+
  ggtitle("Popularity_Rank of the 10 products with the lowest top_sell_rank")

plot_grid(plot_1, plot_2, ncol=1)
}
    
generate13 <- function() {
plot_1 <- ggplot(top_sell_rank_1, aes(x = Top_Sell_Rank, y = Rate , fill = Product)) +
  geom_col()+
  ggtitle("Rate of the 10 products with the highest top_sell_rank ")

plot_2 <- ggplot(top_sell_rank_2, aes(x = Top_Sell_Rank, y = Popularity_Rank , fill = Product)) +
  geom_col()+
  ggtitle("Popularity_Rank of the 10 products with the highest top_sell_rank")
plot_grid(plot_1, plot_2, ncol=1)
}
    
generate14 <- function() {
plot1 <- ggplot(top_product1, aes(x = Rate, y = Product, fill = Party)) +
  geom_col() +
  ggtitle("Top 10 Products by Party by Rate (Descending)")

plot2 <- ggplot(top_product2, aes(x = Rate, y = Product, fill = Party)) +
  geom_col() +
  ggtitle("Top 10 Products by Party by Rate (Ascending)")

plot_grid(plot1, plot2, ncol=1)
    
}
generate15 <- function() {
plot1 <- ggplot(top_sellcuspro_rank_1, aes(x = Top_Sell_Rank, y = Product, fill = Party)) +
  geom_col()+
  ggtitle("Party of 10 products with the highest top_sell_rank")

plot2 <- ggplot(top_sellcuspro_rank_2, aes(x = Top_Sell_Rank, y = Product, fill = Party)) +
  geom_col()+
  ggtitle("Party of 10 products with the lowest top_sell_rank")

plot_grid(plot1, plot2, ncol=1)
}
    
generate16 <- function() {
plot1 <-ggplot(top_rate_pro_popu_Rank_percent_1, aes(x = "", y = percent, fill = Product)) +
  geom_bar(stat = "identity", width = 1) +
geom_text(aes(label = paste0(round(percent, 1), "%")), position = position_stack(vjust = 0.5)) +
coord_polar(theta = "y") +
ggtitle("Rate percentage of the 5 products with the highest Popularity_Rank") +
scale_fill_hue(name = "Product") +
scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
theme(axis.text.y = element_blank(),
axis.ticks.y = element_blank(),
axis.title.y = element_blank())

plot2 <- ggplot(top_rate_pro_popu_Rank_percent_2, aes(x = "", y = percent, fill = Product)) +
  geom_bar(stat = "identity", width = 1) +
geom_text(aes(label = paste0(round(percent, 1), "%")), position = position_stack(vjust = 0.5)) +
coord_polar(theta = "y") +
ggtitle("Rate percentage of the 5 products with the lowest Popularity_Rank") +
scale_fill_hue(name = "Product") +
scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
theme(axis.text.y = element_blank(),
axis.ticks.y = element_blank(),
axis.title.y = element_blank())
plot_grid(plot1, plot2, ncol=1)
}
    
generate17 <- function() {
ggplot(top_cusproduct, aes(x = Product, y = No_of_Orders , fill = Product)) +
  geom_col()+
  ggtitle("10 Products with the highest No_of_Orders at customer level")+
  theme(axis.text.x = element_text(angle = 45,
                                   hjust = 1))
}
    
# Select chart
output$selectedPlot <- renderPlot({
  selectedPlot <- input$PlotType
    
  if (selectedPlot == "For each month (months are combined from 3 years) there will be a total of vouchers (1)") {
    generate1()
  } else if (selectedPlot == "For each year, there will be a total of voucher values (2)") {
    generate2()
  } else if (selectedPlot == "Total Vouchers by Month and Year(3)") {
    generate3()
  } else if (selectedPlot == "Top Party with Qty highest by Month and Year(4)") {
    generate4()
  } else if (selectedPlot == "Top Products with Qty highest by Month and Year(5)") {
    generate5()
  } else if (selectedPlot == "Top 10 Products with Qty highest by Year(6)") {
    generate6()
  } else if (selectedPlot == "Top 10 Products by Year(7)") {
    generate7()
  } else if (selectedPlot == "Top 10 highest Rate products in 2018(8)") {
    generate8()
  } else if (selectedPlot == "Total Rate by Month(9)") {
    generate9()
  } else if (selectedPlot == "Top 10 Products by Rate(10)") {
    generate10()
  } else if (selectedPlot == "Rate percentage of the 5 products with Popularity_Rank(11)") {
    generate11()
  } else if (selectedPlot == "Rate of the 10 products with top_sell_rank(12)") {
    generate12()
  } else if (selectedPlot == "Popularity_Rank of the 10 products with top_sell_rank(13)") {
    generate13()
  } else if (selectedPlot == "Top 10 Products by Party by Rate (14)") {
    generate14()
  } else if (selectedPlot == "Party of 10 products with top_sell_rank(15)") {
    generate15()
  } else if (selectedPlot == "Rate percentage of the 5 products with Popularity_Rank(16)") {
    generate16()
  } else if (selectedPlot == "10 Products with the highest No_of_Orders at customer level(17)") {
    generate17()          
      
  }
})
  
  #Output tab Recommend  
  loginClicked <- reactiveVal(FALSE) # check login
  viewClicked <- reactiveVal(FALSE)  # check view
    
  #Initialize showTabs to FALSE
  showTabs <- FALSE

  observeEvent(input$loginButton, {
    loginClicked(TRUE)
    
    # Validate customer name
    customerName <- input$txt1
    if (!customerName %in% cust_prod_ranking_model$Party) {
      loginClicked(FALSE)
      showModal(modalDialog(
        title = "Error",
        paste("Invalid customer name:", customerName, "Please enter a valid name."),
        footer = tagList(
          actionButton("goBackButton", "Go Back")
        )
      ))
      
      # Set showTabs to false
      showTabs <<- FALSE
      
    } else {
      
      # Set showTabs to true
      showTabs <<- TRUE
      
    }
    
    # Reset viewClicked when logging in again
    viewClicked(FALSE)
  })
  
  observeEvent(input$viewButton, {
    viewClicked(TRUE)
    
    # Validate product name
    productName <- input$txt2
    if (!productName %in% prod_ranking_model$Product) {
      viewClicked(FALSE)
      showModal(modalDialog(
        title = "Error",
        paste("Invalid product name:", productName, "Please enter a valid name."),
        footer = tagList(
          actionButton("goBackButton", "Go Back")
        )
      ))
      
      # Set showTabs to false
      showTabs <<- FALSE
      
    } else {
      
      # Set showTabs to true
      showTabs <<- FALSE 
      
    }
    
    # Reset loginClicked when viewing in again
    loginClicked(FALSE)
  })
  
  observeEvent(input$goBackButton, {
    removeModal()
    
    # Set showTabs to false
    showTabs <<- FALSE
  })
  
  # Check user name vs input
  output$loggedInUI <- renderUI({
    if (loginClicked()) {
      tagList(
        h3(paste("Welcome,", input$txt1)),
        conditionalPanel(
          condition = "input.loginButton && showTabs",
          tabsetPanel(
            tabPanel("Most Popular Products", tableOutput("table1")),
            tabPanel("Top Selling Products", tableOutput("table2")),
            tabPanel("Products you Frequently Purchased", tableOutput("table3")),
            tabPanel("Products you Purchased the Most", tableOutput("table4")),
            tabPanel("Products you may like", tableOutput("table5"))
          )
        )
      )
    } else {
      tagList(
        conditionalPanel(
          condition = "!input.loginButton || !showTabs",
        tabsetPanel(
            tabPanel("Most Popular Products", tableOutput("table1")),
            tabPanel("Top Selling Products", tableOutput("table2"))  
          )
        )
          )
    }
  })
  
  # Check product name vs input
output$viewedInUI <- renderUI({
  if (viewClicked()) {
    tagList(
      h3(paste(input$txt2, "- Price:", prod_ranking_model$Rate[prod_ranking_model$Product == input$txt2])),
      conditionalPanel(
        condition = "input.viewButton",
        tabsetPanel(
          tabPanel("Customers who purchased this product also purchased these", tableOutput("table6"))
        )
      )
    )
  } else {
    tagList(
      conditionalPanel(
        condition = "!input.viewButton",
        NULL
      )
    )
  }
})
     # Display data in tab 1
  output$table1 <- renderTable({
    most_popular_table <- function() {
      most_popular_prods <- prod_ranking_model[order(prod_ranking_model$Popularity_Rank), c("Product", "Rate")][1:10,]
      rownames(most_popular_prods) <- NULL
      return(most_popular_prods)
    }
    most_popular <- most_popular_table()
    head(most_popular)
  })
  
  # Display data in tab 2
  output$table2 <- renderTable({
    top_sell_table <- function() {
      top_sell_prods <- prod_ranking_model[order(prod_ranking_model$Top_Sell_Rank), c("Product", "Rate")][1:10,]
      rownames(top_sell_prods) <- NULL
      return(top_sell_prods)
    }
    top_sell <- top_sell_table()
    top_sell
  })
  
  # Display data in tab 3
  output$table3 <- renderTable({
    cust_most_popular_table <- function(customerName) {
      cust_most_popular_prods <- subset(cust_prod_ranking_model, Party == customerName)[order(subset(cust_prod_ranking_model, Party == customerName)$Popularity_Rank), c("Product", "Rate")][1:10,]
      rownames(cust_most_popular_prods) <- NULL
      return(cust_most_popular_prods)
    }
    if (loginClicked()) {
      cust_most_popular <- cust_most_popular_table(input$txt1)
      cust_most_popular
    }
  })
  
  # Display data in tab 4
  output$table4 <- renderTable({
    cust_top_sell_table <- function(customerName) {
      cust_top_sell_prods <- subset(cust_prod_ranking_model, Party == customerName)[order(subset(cust_prod_ranking_model, Party == customerName)$Top_Sell_Rank), c("Product", "Rate")][1:10,]
      rownames(cust_top_sell_prods) <- NULL
      return(cust_top_sell_prods)
    }
    if (loginClicked()) {
      cust_top_sell <- cust_top_sell_table(input$txt1)
      cust_top_sell
    }
  })
  
  # Display data in tab 5
  output$table5 <- renderTable({
    recommend_products <- function() {
      similar_custs_corr <- cust_correlation_model[input$txt1, ] %>% sort(desc = TRUE)
      prod_by_similar_custs <- data.frame()
  
      for (i in seq_along(similar_custs_corr)) {
        if (names(similar_custs_corr)[i] != input$txt1) {
          cust_top_sell_prods <- cust_prod_ranking_model[cust_prod_ranking_model$Party == names(similar_custs_corr)[i], ]
          cust_top_sell_prods <- cust_top_sell_prods[, c('Product', 'Qty', 'Rate')]
          cust_top_sell_prods <- cust_top_sell_prods[order(cust_top_sell_prods$Product), ]
          rownames(cust_top_sell_prods) <- NULL         
          corr <- as.numeric(similar_custs_corr[1, i])
          cust_top_sell_prods$Qty_Corr <- cust_top_sell_prods$Qty * corr
          prod_by_similar_custs <- rbind(prod_by_similar_custs, cust_top_sell_prods)
        }
      }
  
      prod_by_similar_custs <- prod_by_similar_custs %>%
        group_by(Product) %>%
        summarise(Qty_Corr = sum(Qty_Corr), Rate = max(Rate))
  
      input_cust_top_sell_prods <- subset(cust_prod_ranking_model, Party == input$txt1)
  
      df_merge <- merge(prod_by_similar_custs, input_cust_top_sell_prods %>% select(Product, No_of_Orders), by = "Product", all.x = TRUE)
  
      prod_recommend_to_cust <- subset(df_merge, is.na(No_of_Orders) | No_of_Orders == "") %>%
        arrange(desc(Qty_Corr)) %>%
        select(Product, Rate) %>%
        head(10) %>%
        ungroup() %>%
        mutate(row_id = row_number()) %>%
        select(-row_id)
  
      return(prod_recommend_to_cust)
    }
    if (loginClicked()) {
      recommend <- recommend_products()
      recommend
    }
  })
  
  # Display data in tab 6
  output$table6 <- renderTable({
    get_similar_products <- function(productName) {
      similar_prods_corr <- prod_correlation_model[input$txt2, ]
      similar_prods_corr <- similar_prods_corr[order(similar_prods_corr, decreasing = TRUE)]
      top_10_products <- names(similar_prods_corr)[1:11]
      top_10_products <- data.frame(Product = top_10_products)
  
      similar_prods <- merge(top_10_products, prod_ranking_model[, c('Product', 'Rate')],
                         by = 'Product', all.x = TRUE)
  
      prod_price <- similar_prods[similar_prods$Product == input$txt2, 'Rate'][1]
      input_prod_index <- which(similar_prods$Product == input$txt2)
      similar_prods <- similar_prods[-input_prod_index, ]
  
      similar_prods <- similar_prods %>%
        mutate(row_id = row_number()) %>%
        select(-row_id)
  
      return(similar_prods)
    }
    if (viewClicked()) {
      similar_products <- get_similar_products(input$txt2)
      similar_products
    }
  })
 observeEvent(input$txt2, {
 outputOptions(output, "table6", suspendWhenHidden = FALSE)
  })  
}
shinyApp(ui = ui, server = server)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows
Listening on http://127.0.0.1:5387
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not

"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.